In [1]:
import ccxt
import requests
from binance.client import Client
from binance.exceptions import BinanceAPIException
from datetime import datetime, timedelta
import time
import hashlib
import hmac
import json
import logging
from tqdm import tqdm

In [3]:
from main import *

We are using a delta of 20 weeks
We are using a delta of 3600 seconds
2024-07-18 14:45:09


In [4]:
def get_min_notional(symbol):
    url = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(url)
    data = response.json()
    for s in data['symbols']:
        if s['symbol'] == symbol:
            for filter in s['filters']:
                if filter['filterType'] == 'MIN_NOTIONAL':
                    return float(filter['minNotional'])
    return None

In [11]:
global testnet


In [5]:
symbol="XRPUSDT"

In [15]:
global testnet
testnet
get_account_balances(symbol)

NameError: name 'testnet' is not defined

In [14]:
api_key, api_secret = get_keys("binance_keys.json")
client = Client(api_key, api_secret, testnet=True)

ticker = client.get_symbol_ticker(symbol=symbol)
current_price = float(ticker['price'])

In [19]:
current_price

0.4186

In [23]:
symbol_info = client.get_symbol_info(symbol)
min_notional = None
for filt in symbol_info['filters']:
    if filt['filterType'] == 'MIN_NOTIONAL':
        min_notional = float(filt['minNotional'])

In [28]:
create_hedging_order(symbol,"BUY",12)

INFO:main:Order details saved to order.json


Order placed successfully.


{'symbol': 'XRPUSDT',
 'orderId': 3117535,
 'orderListId': -1,
 'clientOrderId': 'eNfk5sfKEILogtR1IN6dBs',
 'transactTime': 1720186980253,
 'price': '0.00000000',
 'origQty': '12.00000000',
 'executedQty': '12.00000000',
 'cummulativeQuoteQty': '5.04000000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'workingTime': 1720186980253,
 'fills': [{'price': '0.42000000',
   'qty': '12.00000000',
   'commission': '0.00000000',
   'commissionAsset': 'XRP',
   'tradeId': 178377}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [17]:
print(min_notional)

None


In [12]:
calculate_and_place_order(symbol,0)

INFO:main:Current price of XRPUSDT: 0.4159
INFO:main:Share to purchase: 30.38


Current price of XRPUSDT: 0.4159
The purchase order in USD is 12.635041999999999
Create BUY hedging order for: XRPUSDT for 30.38


INFO:main:Order details saved to order.json


Order placed successfully.


In [33]:
print(get_min_notional("XRPUSDT"))

None
